In [1]:
import random
import os
import glob
import shutil

In [2]:
random_country_dir = random.choice(os.listdir("../country_data"))
random_country_dir

'GAB_Gabon'

In [3]:
figures = glob.glob(f"../country_data/{random_country_dir}/figures/*.png", recursive = True)
figures

['../country_data/GAB_Gabon/figures/GAB_CAIT_gases_1.png',
 '../country_data/GAB_Gabon/figures/GAB_GCP_Country_Highlight.png',
 '../country_data/GAB_Gabon/figures/GAB_IEA_1.png',
 '../country_data/GAB_Gabon/figures/GAB_CAIT_lucf_vs_nolucf.png',
 '../country_data/GAB_Gabon/figures/GAB_GCP_1.png',
 '../country_data/GAB_Gabon/figures/GAB_EIA_1.png',
 '../country_data/GAB_Gabon/figures/GAB_CDIAC_1.png',
 '../country_data/GAB_Gabon/figures/GAB_Minx_top20_subsectors.png',
 '../country_data/GAB_Gabon/figures/GAB_UNFCCC_NAI_1.png',
 '../country_data/GAB_Gabon/figures/GAB_CO2_totals.png']

In [4]:
random_fig = random.choice(figures)
random_fig

'../country_data/GAB_Gabon/figures/GAB_Minx_top20_subsectors.png'

In [5]:
shutil.copyfile(random_fig, "../supplemental/random_daily_fig.png",)

'../supplemental/random_daily_fig.png'

In [6]:
print("random_fig path imported")

random_fig path imported
